# Adaboost Lab

## 准备工作
### 环境准备
请确保完成以下依赖包的安装，并且通过下面代码来导入与验证。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


### 数据集准备
我们将使用以下数据集进行 Adaboost 的训练。

该数据集与决策树部分使用的数据集相同，包括 7 个特征以及一个标签“是否适合攻读博士”，涵盖了适合攻读博士的各种条件，如love doing research,I absolutely want to be a college professor等。

请执行下面的代码来加载数据集。


In [ ]:
# read decision_tree_datasets.csv
train_data = pd.read_csv('train_phd_data.csv')
test_data = pd.read_csv('test_phd_data.csv')

# translate lables [0,1] to [-1,1]
# if 0 then -1, if 1 then 1
train_data.iloc[:, -1] = train_data.iloc[:, -1].map({0: -1, 1: 1})
test_data.iloc[:, -1] = test_data.iloc[:, -1].map({0: -1, 1: 1})

## Adaboost (15 pts)

在上一个lab中，你已经成功完成了 Decision Tree 的构建。在本部分，你可以继续沿用上一部分的代码，学习并完成 Adaboost 模型的训练。

在这个 Adaboost 模型中，我们选择了一层决策树作为弱学习器，并使用基尼系数作为分类标准。

请完成以下类的构建以及相应函数的实现：

1. **weakClassifier()**: 我们采用一层决策树，包括 `split()` 和 `predict()`。你可以参考上一次实验中的代码。

2. **Adaboost()**：包括弱学习器的集合，拟合过程 `fit()` 和预测过程 `predict()`。


In [ ]:
class weakClassifier:
    def __init__(self):
        

        self.tree = None 
        self.alpha = None
    
    # here, we use the gini impurity to find the best feature and threshold
    # Note: you need consider sample_weight when computing the gini impurity
        
    def best_split(self, X, y, sample_weight):

        ''' 
            find the best feature and threshold to split the data based on the gini impurity

            Args:
                X: the features of the data
                y: the labels of the data
                sample_weight: the weight of each sample

            Returns:
                best_feature: the best feature to split the data
                best_Series: Series, the data set after splitting
        '''

        # TODO: implement the function to find the best feature and threshold to split the data based on the gini impurity
        n_samples, n_features = X.shape
        best_gini = 1.0
        best_feature = None
        best_Series = pd.Series()

        # Convert X, y, sample_weight into a DataFrame
        data = pd.DataFrame(X)
        data['label'] = y
        data['weight'] = sample_weight

        # Split data based on a feature and its values
        def split_data(data, column):
            split_datas = pd.Series()
            unique_values = data.iloc[:, column].unique()
            for value in unique_values:
                split_datas[value] = data[data.iloc[:, column] == value]
            return split_datas

        # Calculate weighted gini impurity for a subset
        def calculate_weighted_gini(subset):
            total_weight = subset['weight'].sum()
            if total_weight == 0:
                return 0
            sum_squared_probs = sum((subset[subset['label'] == c]['weight'].sum() / total_weight) ** 2 for c in subset['label'].unique())
            return 1 - sum_squared_probs

        # Iterate over all features
        for feature in range(n_features):
            split_series = split_data(data, feature)

            # Calculate weighted gini impurity for each split
            total_gini = 0.0
            for _, subset in split_series.items():
                total_gini += calculate_weighted_gini(subset) * subset['weight'].sum() / data['weight'].sum()

            # Update best split if needed
            if total_gini < best_gini:
                best_gini = total_gini
                best_feature = feature
                best_Series = split_series

        return best_feature, best_Series
    
    
    def fit(self, X, y, sample_weight):
        '''  
            fit the data to the decision tree

            Args:
                X: the features of the data
                y: the labels of the data
                sample_weight: the weight of each sample

            Returns:
                None, but self.tree should be updated
        '''
        best_feature, best_Series = self.best_split(X, y, sample_weight)

        if best_feature is None:
            return 
        
        # TODO: Create the tree as a nested dictionary
        self.tree = {best_feature: {}}
        for value, split_data in best_Series.items():
            # Extract features, labels and weights from the split data
            split_X = split_data.iloc[:, :-2].values  # all columns except last two
            split_y = split_data['label'].values
            split_weight = split_data['weight'].values

            # Determine the most frequent class label in each split
            if len(np.unique(split_y)) == 1:
                self.tree[best_feature][value] = split_y[0]
            else:
                # In a weak classifier (one-level tree), just choose the most common label
                label, count = np.unique(split_y, return_counts=True)
                self.tree[best_feature][value] = label[np.argmax(count)]

        

    def predict(self,x):
        '''  
        predict the label of the data

        Args:
            x: the features of the data
        Return:
            predict_lables: the predict labels of the data
        '''

        # Store the results
        predict_labels = []

        # predict the label of each sample
        for i in range(len(x)):
            sample = x.iloc[i,:]

            # TODO: predict the label of the sample
            match_found = False
            for feature, feature_values in self.tree.items():
                
                if sample[feature] in feature_values:
                    predict_labels.append(1)
                    match_found = True
                    break

            if not match_found:
                # If there is no match, predict -1
                predict_labels.append(-1)

            
        return predict_labels



In [ ]:
class Adaboost:
    
    def __init__(self, n_estimators=10):

        # the number of weak classifier
        self.n_estimators = n_estimators
        # the list of weak classifier
        self.clfs = []
    
    # AdaBoost training process
    def fit(self, X, y):
        n_samples,m_features = X.shape
    
        # initialize weights
        w = np.ones(n_samples)/n_samples

        # for each weak classifier
        for _ in range(self.n_estimators):
            clf = weakClassifier()

            # 1. fit the weak classifier
            clf.fit(X,y,w)

            # TODO: 2. predict the label of the data using the weak classifier
            pred = clf.predict(X)

            # TODO: 3. Calculate errors 
            err = [w[i] * (pred[i] != y[i]) for i in range(n_samples)]
            error = sum(err)

            # TODO:4. Calculate alpha
            alpha = 0.5 * np.log((1 - error) / (error + 1e-10))
            
            # TODO: 5. Update weights
            w = w * np.exp(-alpha * y * np.array(pred))
            
            # normalize to one
            w /= np.sum(w)

            # save classifier and weight
            clf.alpha = alpha
            self.clfs.append(clf)
            

    def predict(self, X):
        '''  
        predict the label of the data
        
        Args:
            X: the features of the data
        Return:
            y_pred: the predict labels of the data
        '''

        #TODO: 1. compute the predict labels of the data using all weak classifiers
        clf_pred = np.array([clf.predict(X) for clf in self.clfs])

        #TODO: 2. compute the weighted sum of the predict labels
        alphas = np.array([clf.alpha for clf in self.clfs])
        weighted_sum = np.dot(alphas, clf_pred)

        #TODO: 3. get the label of the data by sign function (if x>0 return 1, else return -1)
        y_pred = np.sign(weighted_sum)

        return y_pred
        

In [ ]:
adaboost_model = Adaboost(n_estimators=10)
# fit the model
adaboost_model.fit(train_data.iloc[:, :-1], train_data.iloc[:, -1])

# TODO: predict the test data
test_preds = adaboost_model.predict(test_data.iloc[:, :-1])

# TODO: calculate the accuracy of test data
accuracy = ((test_preds == test_data.iloc[:, -1]).sum()) / len(test_data)
print("The accuracy of Adaboost is: ", accuracy)